In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Flatten
from keras.applications.vgg16 import VGG16

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d emmarex/plantdisease

 99% 653M/658M [00:06<00:00, 162MB/s]
100% 658M/658M [00:06<00:00, 101MB/s]


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/plantdisease.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
# Define constants
IMAGE_SIZE = (150, 150)
BATCH_SIZE = 32

In [6]:
import os

dataset_path = '/content/PlantVillage'

# Get the list of subdirectories (classes) in the dataset
classes = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

# Print the class names
for class_name in classes:
    print(class_name)

Tomato_Leaf_Mold
Tomato_Early_blight
Pepper__bell___healthy
Tomato__Tomato_YellowLeaf__Curl_Virus
Potato___healthy
Tomato_Bacterial_spot
Pepper__bell___Bacterial_spot
Tomato__Target_Spot
Potato___Late_blight
Potato___Early_blight
Tomato__Tomato_mosaic_virus
Tomato_Septoria_leaf_spot
Tomato_Spider_mites_Two_spotted_spider_mite
Tomato_healthy
Tomato_Late_blight


Data Augmentation

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load PlantVillage dataset using ImageDataGenerator
#train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)

In [8]:
train_generator = train_datagen.flow_from_directory(
    '/content/PlantVillage',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # since we  have multiple classes
    subset='training'
    #classes=target_folders
)

validation_generator = train_datagen.flow_from_directory(
    '/content/PlantVillage',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
    #classes=target_folders
)

Found 16516 images belonging to 15 classes.
Found 4122 images belonging to 15 classes.


In [9]:
conv_base = VGG16(
    weights = 'imagenet',
    include_top=False,
    input_shape=(150,150,3)
)

58889256/58889256 [==============================] - 0s 0us/step


In [10]:
model = Sequential()

model.add(conv_base)

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dense(len(train_generator.class_indices),activation='softmax'))

In [11]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
# Train the model
history = model.fit(train_generator,
                    epochs=5,
                    validation_data=validation_generator)

Epoch 1/5
517/517 [==============================] - 201s 348ms/step - loss: 2.0716 - accuracy: 0.3260 - val_loss: 1.6880 - val_accuracy: 0.4505
Epoch 2/5
517/517 [==============================] - 162s 313ms/step - loss: 1.2965 - accuracy: 0.5652 - val_loss: 1.0598 - val_accuracy: 0.6361
Epoch 3/5
517/517 [==============================] - 162s 313ms/step - loss: 0.9638 - accuracy: 0.6726 - val_loss: 0.8020 - val_accuracy: 0.7251
Epoch 4/5
517/517 [==============================] - 161s 311ms/step - loss: 0.7709 - accuracy: 0.7397 - val_loss: 0.7211 - val_accuracy: 0.7475
Epoch 5/5
517/517 [==============================] - 177s 342ms/step - loss: 0.6231 - accuracy: 0.7871 - val_loss: 0.6228 - val_accuracy: 0.7872


In [13]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

In [19]:
# Function to preprocess an individual image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Function to make predictions on an individual image
def predict_image(img_path):
    processed_img = preprocess_image(img_path)
    predictions = model.predict(processed_img)
    print(classes[np.argmax(predictions)])
    print(np.argmax(predictions))
   # decoded_predictions = decode_predictions(predictions, top=3)[0]
    #return decoded_predictions

# Test the model on an example image
img_path = '/content/Pepper__bell___Bacterial_spot.jpg'
predictions = predict_image(img_path)

1/1 [==============================] - 0s 18ms/step
Potato___Early_blight
9
